In [1]:
import sqlite3
import json

In [2]:
!rm sordi.coco

In [3]:
import numpy as np                                 # (pip install numpy)

def create_sub_mask_annotation(image_id, category_id, annotation_id, bbox, is_crowd):
   
    annotation = {
        'iscrowd': is_crowd,  # dont need
        'image_id': image_id, # pointing to an image entry (see below)
        'category_id': category_id,   # that is the label of the object, see id field from "category" below
        'id': annotation_id,   
        'bbox': bbox    # the bbox as 4-tuple
    }

    return annotation

def create_image_entry(image_id, file_name, width, height):
    
    image_entry = {
        'id': image_id,   # unique, annotations "image_id" field are pointing to this one
        'width': width,   # framesize, annotations are in pixelcoords
        'height': height,
        'file_name': file_name
    }
    
    return image_entry

def create_category_entry(obj_id, obj_name):
    
    category_entry = {
        'id': obj_id,
        'name': obj_name
    }
    
    return category_entry
    

In [4]:
annotations = []
images = []
categories = []

In [5]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def iterate_through_db(conn):
    category = {}
    
    sql = '''SELECT ROWID,* FROM FRAME ''' # ROWID is image_id below
    cur = conn.cursor()
    cur.execute(sql)
    
    annotation_id = 0
    
    for r in cur:
        (image_id, fname, label_json, w, h, uncertainty) = r  

        images.append(create_image_entry(image_id, fname, w, h))
        
        for lj in json.loads(label_json):
            (l,t,r,b) = (lj['Left'], lj['Top'], lj['Right'], lj['Bottom'])
            
            bbox = (l, t, r-l, b-t)
            annotations.append(create_sub_mask_annotation(image_id, lj['ObjectClassId'], annotation_id , bbox, 0))
            
            ++annotation_id
            
            if lj['ObjectClassId'] in category: # category.keys()
                pass
            else:
                category[lj['ObjectClassId']] = lj['ObjectClassName']
           
    for k in category:
        categories.append(create_category_entry(k, category[k]))
        
    print(categories)

In [6]:
conn = create_connection("SORDI.sqlite")

iterate_through_db(conn)

[{'id': 2000, 'name': 'bicycle'}, {'id': 1040, 'name': 'cabinet'}, {'id': 1110, 'name': 'dolly'}, {'id': 2010, 'name': 'forklift'}, {'id': 1120, 'name': 'jack'}, {'id': 1030, 'name': 'locker'}, {'id': 1100, 'name': 'pallet'}, {'id': 2050, 'name': 'str'}, {'id': 5010, 'name': 'fire_extinguisher'}, {'id': 1070, 'name': 'cardboard_box'}, {'id': 1011, 'name': 'l_klt_4147'}, {'id': 1003, 'name': 'stillage_open'}, {'id': 1002, 'name': 'stillage_close'}, {'id': 1013, 'name': 'l_klt_8210'}, {'id': 1012, 'name': 'l_klt_6147'}, {'id': 1135, 'name': 'spring_post'}, {'id': 4000, 'name': 'exit_sign'}]


In [7]:
coco = json.dumps({'info': {"description": "SORDI 2022"}, 'categories': categories, 'annotations': annotations, 'images': images}, indent=4)

with open("sordi.coco", 'w') as file:
    file.write(coco)